<a href="https://colab.research.google.com/github/fpd4fv/ds3002/blob/gh-pages/spacy_full_cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==3.1.1
!pip install spacy-transformers
!python -m spacy download en_core_web_trf


     |████████████████████████████████| 6.4 MB 6.9 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 457 kB 47.7 MB/s 
     |████████████████████████████████| 10.1 MB 32.6 MB/s 
     |████████████████████████████████| 653 kB 57.8 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uninstall

In [2]:
import spacy
import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers

# Storing docs in binary format
from spacy.tokens import DocBin

In [3]:
from zipfile import ZipFile

In [5]:
#Uploading model folder to colab
file_name = '/content/nlp_model_forzip.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')


Done


In [6]:
#Reading in trained model

#Test the data from the best model
#nlp = spacy.load("output_updated/model-best")
nlp = spacy.load("/content/nlp_model_forzip")

text = "Australia’s largest airline temporarily lays off 2,500 employees"
demo = nlp(text)
print(demo.cats)


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


{'positive': 0.0006150818080641329, 'negative': 0.9989302754402161, 'neutral': 0.00045462805428542197}


In [7]:
#Bringing in Twitter API functionality
import tweepy
from tweepy import Stream
import datetime

In [17]:
#Twitter devel credentials
access_token="1433202678818017283-Tz047y4zcoa9ivP7r0UHlmuko2MqKd"
access_token_secret="OeubpAnY4u0yPp0Pckcor6589hNB9gU06dojxRGf2jw8A"
API_key="qfOhwhutCVNW80q0C4WnXBdPn"
API_secret_key="HAmpbaSuZNWgaAoLi7092QjsH2wS6jsXH991e6fjAJGc4KnIaI"

auth = tweepy.OAuthHandler(API_key, API_secret_key)
auth.set_access_token(access_token, access_token_secret)


In [18]:
api = tweepy.API(auth, wait_on_rate_limit=True)


In [35]:
### Getting user and printing their tweets
def user_tweets(twitter_handle, number_of_tweets_wanted):
  user = api.get_user(screen_name = twitter_handle)
  user_tweets = api.user_timeline(screen_name=user)
  #setting user_tweets limited to desired number in order to not hit twitter developer rate limits
  user_tweets = user_tweets[0:number_of_tweets_wanted]
  tweet_ids = [tweet.id for tweet in user_tweets]
  tweet_user = [twitter_handle]*len(user_tweets)
  tweet_dates = [tweet.created_at for tweet in user_tweets]
  tweet_texts = [tweet.text for tweet in user_tweets]
  #Producing df with these values
  user_tweets_dict = {
    'Tweet ID': tweet_ids,
    'Tweet Poster': tweet_user,
    'Tweet Date': tweet_dates,
    'Tweet Text': tweet_texts,
    }
  user_tweets_df = pd.DataFrame(user_tweets_dict)
  print(user_tweets_df.head())
  return user_tweets_df

  

In [36]:
user_tweets('zlj517', 50)

              Tweet ID     Tweet Poster          Tweet Date  \
0  1520101772936851456  ParkerDriscoll2 2022-04-29 18:04:39   
1  1519467485128531969  ParkerDriscoll2 2022-04-28 00:04:13   

              Tweet Text  
0  @ds3002_tweet_analyze  
1           test tweet 3  


,Tweet ID,Tweet Poster,Tweet Date,Tweet Text
0,1520101772936851456,ParkerDriscoll2,2022-04-29 18:04:39,@ds3002_tweet_analyze
1,1519467485128531969,ParkerDriscoll2,2022-04-28 00:04:13,test tweet 3


In [38]:
#Function to see if tweet is talking about the US
relevant_to_US_words = ['US','U.S.','#US',"US'", 'United States','States',"States'",'America','Biden','NATO','#NATO']


In [42]:
### Getting user tweets about the U.S. and assembling the tweets in a df
def user_tweets_about_US(twitter_handle):
  user = api.get_user(screen_name = twitter_handle)
  user_tweets = api.user_timeline(screen_name=user, count=100)
  relevant_tweets = []
  for tweet in user_tweets:
    for word in tweet:
      if word in relevant_to_US_words:
        relevant_tweets.append(tweet)

  tweet_ids = [tweet.id for tweet in relevant_tweets]
  tweet_user = [twitter_handle]*len(relevant_tweets)
  tweet_dates = [tweet.created_at for tweet in relevant_tweets]
  tweet_texts = [tweet.text for tweet in relevant_tweets]
  #Producing df with these values
  relevant_tweets_dict = {
    'Tweet ID': tweet_ids,
    'Tweet Poster': tweet_user,
    'Tweet Date': tweet_dates,
    'Tweet Text': tweet_texts,
    }
  relevant_tweets_df = pd.DataFrame(relevant_tweets_dict)
  print(relevant_tweets_df.head())
  return relevant_tweets_df

In [43]:
user_tweets_about_US('zlj517')
#NEED TO ADDRESS RATE LIMIT CONCERNS

TweepError: ignored